In [232]:
import os
import re
import nltk
import string
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import  WordNetLemmatizer
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [233]:
data = pd.read_csv("spam.csv", encoding="latin-1")

In [234]:
data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [235]:
data.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4"], axis = 1, inplace = True)

In [13]:
data

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [236]:
stop_words = set(stopwords.words('english'))

In [336]:
def preprocess_text(text):
    text=text.lower()
    text = re.sub(r'[^\w\s\d\\.\\!\\?]','', text) 
    sent = [sent.strip() for sent in nltk.sent_tokenize(text)]
    sent = [sentence.replace("\n","") for sentence in sent]
    return sent

#### Векторизация + модель

In [337]:
vec = TfidfVectorizer(tokenizer=lambda x:preprocess_text(x))

In [338]:
matrix = vec.fit_transform(data["v2"].values)

In [383]:
model = DBSCAN(eps=1.0, min_samples=4, n_jobs=-1)

In [384]:
model.fit(matrix)

DBSCAN(algorithm='auto', eps=1.0, leaf_size=30, metric='euclidean',
       metric_params=None, min_samples=4, n_jobs=-1, p=None)

In [385]:
clusters = model.labels_.tolist()

In [216]:
#clusters

In [386]:
data["clusters"] = model.fit_predict(matrix)

In [387]:
data

,v1,v2,clusters,tokens,sentiment_score
0,ham,"Go until jurong point, crazy.. Available only ...",0,"[go, jurong, point, crazy, available, bugis, n...",1
1,ham,Ok lar... Joking wif u oni...,0,"[ok, lar, joking, wif, u, oni]",0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,0,"[free, entry, 2, wkly, comp, win, fa, cup, fin...",2
3,ham,U dun say so early hor... U c already then say...,0,"[u, dun, say, early, hor, u, c, already, say]",0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,"[nah, think, goes, usf, lives, around, though]",0
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,0,"[2nd, time, tried, 2, contact, u, u, å, 750, p...",2
5568,ham,Will Ì_ b going to esplanade fr home?,0,"[ì_, b, going, esplanade, fr, home]",0
5569,ham,"Pity, * was in mood for that. So...any other s...",-1,"[pity, mood, suggestions]",-1
5570,ham,The guy did some bitching but I acted like i'd...,0,"[guy, bitching, acted, like, interested, buyin...",2


In [388]:
data.clusters.value_counts()

 0    4400
-1    1152
 2      12
 1       8
Name: clusters, dtype: int64

Так, видим, что у нас 1152 выброса. Попробуем разобраться.

In [368]:
outlier = data[data.clusters==-1]

In [369]:
outlier

,v1,v2,clusters,tokens,sentiment_score
6,ham,Even my brother is not like to speak with me. ...,-1,"[even, brother, like, speak, treat, like, aids...",2
7,ham,As per your request 'Melle Melle (Oru Minnamin...,-1,"[per, request, melle, melle, oru, minnaminungi...",0
10,ham,I'm gonna be home soon and i don't want to tal...,-1,"[gon, na, home, soon, want, talk, stuff, anymo...",1
11,spam,"SIX chances to win CASH! From 100 to 20,000 po...",-1,"[six, chances, win, cash, 100, 20, 000, pounds...",1
15,spam,"XXXMobileMovieClub: To use your credit, click ...",-1,"[xxxmobilemovieclub, use, credit, click, wap, ...",0
...,...,...,...,...,...
5551,ham,Wen did you get so spiritual and deep. That's ...,-1,"[wen, get, spiritual, deep, great]",2
5552,ham,Have a safe trip to Nigeria. Wish you happines...,-1,"[safe, trip, nigeria, wish, happiness, soon, c...",2
5560,ham,Anything lor. Juz both of us lor.,-1,"[anything, lor, juz, us, lor]",0
5569,ham,"Pity, * was in mood for that. So...any other s...",-1,"[pity, mood, suggestions]",-1


In [370]:
outlier["v1"].value_counts()

ham     1003
spam     149
Name: v1, dtype: int64

Больше всего выбросов в обычных сообщениях(хотя изначально их больше)

#### sentiment sore

In [371]:
f1 = open("positive-words.txt")
f2 = open("negative-words.txt")
pos =[]
neg =[]

for line in f1:
    line = line.strip()
    pos.append(line)
for line in f2:
    line = line.strip()
    neg.append(line)

In [372]:
def tokenize(text):
    text = re.sub(r'[^\w\d]',' ', text) 
    tokens = [word.lower() for word in nltk.word_tokenize(text)]
    tokens = [w for w in tokens if w not in stop_words]
    return tokens
        

In [373]:
data["tokens"] = data["v2"].apply(lambda x: tokenize(x))

In [374]:
data["sentiment_score"] = data["tokens"].apply(lambda x: (sum(1 for w in x if w in pos)) + (sum(-1 for w in x if w in neg)))

In [375]:
outlier = data[data.clusters==-1]

In [376]:
outlier

,v1,v2,clusters,tokens,sentiment_score
6,ham,Even my brother is not like to speak with me. ...,-1,"[even, brother, like, speak, treat, like, aids...",2
7,ham,As per your request 'Melle Melle (Oru Minnamin...,-1,"[per, request, melle, melle, oru, minnaminungi...",0
10,ham,I'm gonna be home soon and i don't want to tal...,-1,"[gon, na, home, soon, want, talk, stuff, anymo...",1
11,spam,"SIX chances to win CASH! From 100 to 20,000 po...",-1,"[six, chances, win, cash, 100, 20, 000, pounds...",1
15,spam,"XXXMobileMovieClub: To use your credit, click ...",-1,"[xxxmobilemovieclub, use, credit, click, wap, ...",0
...,...,...,...,...,...
5551,ham,Wen did you get so spiritual and deep. That's ...,-1,"[wen, get, spiritual, deep, great]",2
5552,ham,Have a safe trip to Nigeria. Wish you happines...,-1,"[safe, trip, nigeria, wish, happiness, soon, c...",2
5560,ham,Anything lor. Juz both of us lor.,-1,"[anything, lor, juz, us, lor]",0
5569,ham,"Pity, * was in mood for that. So...any other s...",-1,"[pity, mood, suggestions]",-1
